**`ICUBAM`: ICU Bed Availability Monitoring and analysis in the *Grand Est région* of France during the COVID-19 epidemic.**

https://doi.org/10.1101/2020.05.18.20091264

Python notebook for the sir-like modeling (see Section IV.1 of the main paper).

(iii) compute and save samples from the posterior distribution P(parameters|data)

In [1]:
import numpy as np
import pandas as pd
import os.path as op

import pymc3 as pm 
from theano.compile.ops import as_op
import theano.tensor as T

from collections import namedtuple

import model_icubam as micu

np.random.seed(13)

In [2]:
data_pop = pd.read_csv(op.join('data', 'pop_dep_2020.csv'), delimiter='\t')

#Fig. 11 and Fig. 17 (right) are realized using first_date = '2020-03-19' and last_date = '2020-04-27'.
first_date = '2020-03-19'
last_date = '2020-04-27'

data = pd.read_csv(op.join('data', 'all_bedcounts_2020-05-04_11h02.csv'), index_col=0)              
data = data.groupby(['date', 'department']).sum().reset_index()
data = data[data.date >= first_date]
data = data[data.date <= last_date]

In [3]:
df_params_hat = pd.read_csv(op.join(micu.model_path, 'params_hat_{}.csv'.format(last_date)), index_col=0)              
sites = df_params_hat.dep.values
params_hat = df_params_hat[[col for col in df_params_hat.columns if not col.startswith('dep')]].values

n_sites = len(sites)

depname2depid = {'Ardennes':8, 'Aube':10, 'Marne':51, 'Haute-Marne':52,
                 'Meurthe-et-Moselle':54, 'Meuse':55, 'Moselle':57, 'Bas-Rhin':67,
                 'Haut-Rhin':68, 'Vosges':88}

In [4]:
micu.make_dir(op.join(micu.model_path, 'samples'))

compute_model = micu.compute_model_seir

In [5]:
draws = 2500
chains = 4
tune = 1000

In [6]:
def get_samples(params_star, icu_data, exit_data, sigma, pop):
    n_days = icu_data.size
    
    @as_op(itypes=[T.lscalar, T.dscalar, T.dscalar, T.dscalar, 
                   T.dscalar, T.dscalar, T.dscalar, T.dscalar],
           otypes=[T.dvector, T.dvector])
    def compute_model_pm(n_days_pre, alpha_e, beta, wei, wiout, wic, wcc, wcx):
        params = [n_days_pre, alpha_e, beta, wei, wiout, wic, wcc, wcx]
        c, x = compute_model(pop, params, n_days)
        return c, x

    with pm.Model() as model:
        n_days_pre = T.as_tensor_variable(np.int64(params_star[0]))
        alpha_e = pm.Uniform('alpha_e', 0.0, 1.0)
        beta = pm.Uniform('beta', 0.0, 1.0)
        wei = pm.Uniform('wei', 0.0, 1.0)
        wiout = pm.Uniform('wiout', 0.0, 1.0)
        wic = pm.Uniform('wic', 0.0, 1.0)
        wcc = pm.Uniform('wcc', 0.0, 1.0)
        wcx = pm.Uniform('wcx', 0.0, 1.0)

        c, x = compute_model_pm(n_days_pre, alpha_e, beta, wei, wiout, wic, wcc, wcx)

        icu = pm.Normal('icu', mu=c, sigma=sigma, 
                        observed=icu_data)
        exit = pm.Normal('out', mu=x, sigma=sigma, 
                        observed=exit_data)
        
        step = pm.Slice([alpha_e, beta, wei, wiout, wic, wcc, wcx])
        start = {'alpha_e': params_star[1], 
         'beta': params_star[2], 
         'wei': params_star[3], 
         'wiout': params_star[4], 
         'wic': params_star[5], 
         'wcc': params_star[6], 
         'wcx': params_star[7]}

        trace = pm.sample(draws=draws, start=start, tune=tune, chains=chains,
                          step=step, model=model)
        
        df_trace = pm.trace_to_dataframe(trace)
        logp = model.logp
        samples_logp = np.array([logp(trace.point(i,chain=c)) for c in trace.chains for i in range(len(trace))])
        df_trace['logp'] = samples_logp
    return df_trace

In [7]:
for k, dep in enumerate(sites):
    print('*'*10, dep, '*'*10)
    params_star = params_hat[k].copy()
    params_star[1:] = np.clip(params_star[1:], a_min=1e-10, a_max=(1-1e-10))
    condition = data.department==dep

    n_days = data[condition].date.size
    data[condition]['n_covid_occ'].size
    pop = data_pop[data_pop.dep=='{}'.format(depname2depid[dep])]['pop'].values[0]

    # evaluate noise variance for this departement
    c, x = compute_model(pop, params_star, n_days)
    sigma = .5*(np.std(data[condition]['n_covid_occ'].values - c) 
                + np.std(data[condition]['n_covid_deaths'].values
                         +data[condition]['n_covid_healed'].values - x))

    df_samples = get_samples(params_star, 
                             data[condition]['n_covid_occ'].values,
                             (data[condition]['n_covid_deaths'].values
                              +data[condition]['n_covid_healed'].values),
                             sigma, pop)
    
    df_samples.insert(0, 'n_days_pre', params_star[0])
    df_samples.to_csv(op.join(micu.model_path, 
            'samples', 'samples_{}_{}.csv'.format(dep.lower(), last_date)))

********** Ardennes **********


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [wcx]
>Slice: [wcc]
>Slice: [wic]
>Slice: [wiout]
>Slice: [wei]
>Slice: [beta]
>Slice: [alpha_e]
Sampling 4 chains, 0 divergences: 100%|██████████| 14000/14000 [02:04<00:00, 112.86draws/s]
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


********** Aube **********


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [wcx]
>Slice: [wcc]
>Slice: [wic]
>Slice: [wiout]
>Slice: [wei]
>Slice: [beta]
>Slice: [alpha_e]
Sampling 4 chains, 0 divergences: 100%|██████████| 14000/14000 [01:45<00:00, 133.12draws/s]
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


********** Bas-Rhin **********


...

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [wcx]
>Slice: [wcc]
>Slice: [wic]
>Slice: [wiout]
>Slice: [wei]
>Slice: [beta]
>Slice: [alpha_e]
Sampling 4 chains, 0 divergences: 100%|██████████| 14000/14000 [02:15<00:00, 103.03draws/s]
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


********** Moselle **********


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [wcx]
>Slice: [wcc]
>Slice: [wic]
>Slice: [wiout]
>Slice: [wei]
>Slice: [beta]
>Slice: [alpha_e]
Sampling 4 chains, 0 divergences: 100%|██████████| 14000/14000 [02:34<00:00, 90.76draws/s]
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


********** Vosges **********


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [wcx]
>Slice: [wcc]
>Slice: [wic]
>Slice: [wiout]
>Slice: [wei]
>Slice: [beta]
>Slice: [alpha_e]
Sampling 4 chains, 0 divergences: 100%|██████████| 14000/14000 [01:57<00:00, 119.03draws/s]
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
